In [ ]:
#!/usr/bin/env python3
# coding:utf-8
# author: yuanbin.myb@alibaba-inc.com

import numpy as np
import rasterio
import sys
import fiona
import rasterio.transform as T
from rasterio import features
from shapely.geometry import shape, Polygon, MultiPolygon, mapping, LineString
import json


def transform_polygon(polygon, transform):
    cols, rows = [], []
    for pt in polygon:
        cols.append(pt[0])
        rows.append(pt[1])
    xs, ys = T.xy(transform, rows=rows, cols=cols)

    coordinates = []
    for x, y in zip(xs, ys):
        coordinates.append((x, y))

    return coordinates

class ImageUtils(object):

    """docstring for ImageUtils"""
    def __init__(self):
        super(ImageUtils, self).__init__()
        print("ImageUtils constructed.")

    @staticmethod    
    def _image_to_tif(src, des):
          s_data = rasterio.open(src)
          w = s_data.width
          h = s_data.height
          profile = s_data.profile
          print(profile)

          # fake geo transform
          geotransform = rasterio.Affine(5.364453124996615e-06, 0.0, 116.1529541,0.0, -5.364453124999391e-06, 40.0836182)       
          profile.update(driver='GTiff', dtype='uint8', crs='EPSG:4490', transform=geotransform)
          with rasterio.open(des, 'w', **profile) as dst:
              dst.write(s_data.read())  
              print("done.")

    @staticmethod
    def _affine_transformation(src, des):

      #open json file from third-part platform
      fjson = open(src,)

      # geo feature
      schema = {
        'geometry': 'Polygon',
        'properties': {'group_id': 'int', 'label': 'str'},
      }

      # shape file
      with fiona.open(des, 'w', crs='EPSG:4490', driver='ESRI Shapefile', schema=schema) as c:

        for s in json.load(fjson)['shapes']:
              # do affine transform
              geotransform = rasterio.Affine(5.364453124996615e-06, 0.0, 116.1529541,0.0, -5.364453124999391e-06, 40.0836182)
              p = transform_polygon(s['points'], geotransform)
              poly = Polygon(p)
              print(int(s['group_id']))
              print(str(s['label']))

              c.write({
                  'geometry': mapping(poly),
                  'properties': {'group_id': int(s['group_id']), 'label': str(s['label'])}
              })

      fjson.close()
      print('done.')

    def destory():
            print("Destory.")    
    


if __name__ == '__main__':
  if len(sys.argv) != 3:
    print('Usage: Need 2 inputs: src_tif,des_tif')
    exit(1)

  #源图(JPG)
  src = sys.argv[1]

  #目标图(Tiff)
  des = sys.argv[2]

  utils  = ImageUtils()

  # JPG -> tif
  utils._image_to_tif(src, des)

  # json -> shp
  # utils._affine_transformation(src, des)